In [ ]:
# ===== SABER — Bloco 1: setup (bitstream + MMIO + reset inicial) =====
# Rode este bloco 1x após ligar a placa ou recompilar o bitstream.

import time
from pynq import Overlay, MMIO

# ---------- CONFIG GERAL ----------
BIT  = "/home/xilinx/jupyter_notebooks/SABER.bit"
READ_PERIOD_S = 0.10            # 100 ms entre leituras

# Offsets do AXI GPIO (2 canais)
CH1_DATA, CH1_TRI = 0x00, 0x04   # CH1 = GO
CH2_DATA, CH2_TRI = 0x08, 0x0C   # CH2 = RESET
RESET_ACTIVE_HIGH = True         # True: 1=reset; False: 0=reset ativo-baixo

print("Loading bitstream…")
ol = Overlay(BIT)
ol.download()
print("IPs found:", list(ol.ip_dict.keys()))

def find_ip_named(substr, ip_dict):
    for k in ip_dict:
        if substr.lower() in k.lower():
            return k
    return None

def find_first(ip_dict, *candidates):
    for c in candidates:
        k = find_ip_named(c, ip_dict)
        if k:
            return k
    return None

# --- MMIO do registrador de 1792b (wifi_pack_reg / reg_v3) ---
ip_name = find_first(ol.ip_dict, "wifi_pack_reg", "reg_v3", "wifi_pack")
assert ip_name, f"IP do registrador (wifi_pack_reg/reg_v3) não encontrado. IPs: {list(ol.ip_dict.keys())}"
info_reg = ol.ip_dict[ip_name]
mmio_reg = MMIO(info_reg["phys_addr"], info_reg["addr_range"])
print("Reading from:", ip_name, hex(info_reg["phys_addr"]))

# --- MMIO do AXI GPIO (GO/RESET) ---
gpio_name = find_first(ol.ip_dict, "axi_gpio", "gpio")
if gpio_name:
    info_gpio = ol.ip_dict[gpio_name]
    mmio_gpio0 = MMIO(info_gpio["phys_addr"], info_gpio["addr_range"])
    print("GPIO via:", gpio_name, hex(info_gpio["phys_addr"]))
else:
    # fallback no endereço clássico do AXI GPIO
    GPIO0_BASE  = 0x41200000
    GPIO_RANGE  = 0x10000
    mmio_gpio0 = MMIO(GPIO0_BASE, GPIO_RANGE)
    print("GPIO via endereço fixo:", hex(GPIO0_BASE))

# Direções: saída nos canais usados
mmio_gpio0.write(CH1_TRI, 0x00000000)  # CH1 (GO)    -> out
mmio_gpio0.write(CH2_TRI, 0x00000000)  # CH2 (RESET) -> out

# -------- Helpers globais (usados no Bloco 2) --------
def _gpio_read(mmio, off):  return mmio.read(off)
def _gpio_write(mmio, off, val): mmio.write(off, val & 0xFFFFFFFF)

def set_go(level: int):
    """GO no bit0 do CH1 do AXI GPIO."""
    v = _gpio_read(mmio_gpio0, CH1_DATA)
    if level: v |=  (1 << 0)
    else:     v &= ~(1 << 0)
    _gpio_write(mmio_gpio0, CH1_DATA, v)

def set_reset(level: int):
    """RESET no bit0 do CH2 do AXI GPIO."""
    want = 1 if level else 0
    if not RESET_ACTIVE_HIGH:
        want ^= 1
    v2 = _gpio_read(mmio_gpio0, CH2_DATA)
    if want: v2 |=  (1 << 0)
    else:    v2 &= ~(1 << 0)
    _gpio_write(mmio_gpio0, CH2_DATA, v2)

def pulse_reset(ms: int = 50):
    set_reset(1); time.sleep(ms/1000.0); set_reset(0)

def pulse_go(ms: int = 20):
    set_go(1); time.sleep(ms/1000.0); set_go(0)

def read_56_words():
    # 56 x 32-bit = 1792 bits (words 0..55)
    return [mmio_reg.read(i*4) for i in range(56)]

def words_to_hex_be(words):
    # Concatena cada word (32b) em 8 hex (big-endian por word)
    return "".join(f"{w:08x}" for w in words)

# ---------- Reset inicial ----------
print("[SYS] Reset inicial…")
pulse_reset(50)
set_go(0)

print("Setup concluído. Agora rode o Bloco 2 para conectar e operar.")
